In [20]:
from sqlalchemy import create_engine
import pandas as pd
import json
import numpy as np

In [21]:
# read the json file to python dict
def get_postgres_credentials(path='config.json'):
    with open(path, 'r') as file:
        config = json.load(file)
    return config['postgres']

# connect to postgres using create_engine
def connect_to_postgres():
    credentials = get_postgres_credentials()
    connection_string = (
        f"postgresql://{credentials['user']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database']}"
    )
    engine = create_engine(connection_string)
    connection = engine.connect()
    return connection

In [22]:
df = pd.read_csv('Datasets/tickets.csv')

In [23]:
df.head()

,Created Date,Ticket No.,Category,Sub-Category,Subject,Status,Department,Urgency,Date Resolved,Isolation
0,2023-01-05,#INC-5497977,ITD HQ Support,Network Concern,Network Connection Installation,Resolved,ITD,NaN,2023-01-05,NaN
1,2023-01-05,#INC-5498082,ITD HQ Support,DPC Email,Transferring of Thunderbird Account,Resolved,FINANCE,NaN,2023-01-05,NaN
2,2023-01-09,#INC-5523025,ITD HQ Support,Network Concern,Network Connection Assistance,Resolved,NaN,NaN,2023-01-09,NaN
3,2023-01-09,#INC-5509790,ITD HQ Support,PC/Printer Concern,Installation of Autocad and Sketchup,Resolved,DNC,NaN,2023-01-11,NaN
4,2023-01-09,#INC-5526435,HQ/District Office,PC/Printer Concern,Wifi connection and printer connection in Visa...,Resolved,OPERATIONS,NaN,2023-01-10,NaN


In [24]:
df.drop(columns=['Category'], inplace=True)

In [25]:
df.rename(columns={"Created Date":"created_date", "Ticket No.": "ticket_id","Sub-Category": "category_name", "Isolation": "remarks", "Date Resolved": "resolved_date", "Department": "department_name"}, inplace=True)

In [26]:
df['ticket_id'] = df['ticket_id'].str.strip('#INC-')

In [27]:
df['Urgency'] = df['Urgency'].apply(lambda x: np.random.choice(['Urgent', 'High', 'Medium', 'Low']) if pd.isnull(x) else x)

In [28]:
df[df['ticket_id'].duplicated() == True]

,created_date,ticket_id,category_name,Subject,Status,department_name,Urgency,resolved_date,remarks


In [29]:
df['ticket_id'] = df['ticket_id'].astype('int')
df['created_date'] = pd.to_datetime(df['created_date'])
df['resolved_date'] = pd.to_datetime(df['resolved_date'])

In [34]:
df['Urgency'] = df['Urgency'].apply(lambda x: np.random.choice(['Urgent', 'High', 'Medium', 'Low']) if pd.isnull(x) else x)

In [36]:
unique_dept = df['department_name'].dropna().unique()
df['department_name'] = df['department_name'].fillna(pd.Series(np.random.choice(unique_dept, size=len(df.index))))

In [33]:
df

,created_date,ticket_id,category_name,Subject,Status,department_name,Urgency,resolved_date,remarks
0,2023-01-05,5497977,Network Concern,Network Connection Installation,Resolved,ITD,Medium,2023-01-05,NaN
1,2023-01-05,5498082,DPC Email,Transferring of Thunderbird Account,Resolved,FINANCE,Urgent,2023-01-05,NaN
2,2023-01-09,5523025,Network Concern,Network Connection Assistance,Resolved,NaN,Urgent,2023-01-09,NaN
3,2023-01-09,5509790,PC/Printer Concern,Installation of Autocad and Sketchup,Resolved,DNC,Medium,2023-01-11,NaN
4,2023-01-09,5526435,PC/Printer Concern,Wifi connection and printer connection in Visa...,Resolved,OPERATIONS,Medium,2023-01-10,NaN
...,...,...,...,...,...,...,...,...,...
438,2024-02-15,7035163,Email Concern,Forgot password,Open,BDD,Medium,NaT,NaN
439,2024-02-15,7037058,Software/App Installation,Excel Problem - file can't be saved,Resolved,LEGAL,Low,2024-02-15,File save in autorecovery
440,2024-02-16,7039343,DPC Module Remote Desktop,DPC Remote Desktop - Bacolod 02/16/2024,Resolved,FINANCE,Low,2024-02-16,coordinate to HQ group to restart the module
441,2024-02-16,7039580,DPC Module Remote Desktop,Unable to access remote module IP 192.168.20.1...,Resolved,FINANCE,Low,2024-02-16,coordinate to HQ group to restart the module


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   created_date     443 non-null    datetime64[ns]
 1   ticket_id        443 non-null    int32         
 2   category_name    443 non-null    object        
 3   Subject          443 non-null    object        
 4   Status           443 non-null    object        
 5   department_name  379 non-null    object        
 6   Urgency          67 non-null     object        
 7   resolved_date    439 non-null    datetime64[ns]
 8   remarks          271 non-null    object        
dtypes: datetime64[ns](2), int32(1), object(6)
memory usage: 29.5+ KB


In [ ]:
df = df.drop_duplicates()

In [ ]:
df

,created_date,ticket_id,category_name,Subject,Status,department_name,Urgency,resolved_date,remarks
0,2023-01-05,5497977,Network Concern,Network Connection Installation,Resolved,ITD,NaN,2023-01-05,NaN
1,2023-01-05,5498082,DPC Email,Transferring of Thunderbird Account,Resolved,FINANCE,NaN,2023-01-05,NaN
2,2023-01-09,5523025,Network Concern,Network Connection Assistance,Resolved,NaN,NaN,2023-01-09,NaN
3,2023-01-09,5509790,PC/Printer Concern,Installation of Autocad and Sketchup,Resolved,DNC,NaN,2023-01-11,NaN
4,2023-01-09,5526435,PC/Printer Concern,Wifi connection and printer connection in Visa...,Resolved,OPERATIONS,NaN,2023-01-10,NaN
...,...,...,...,...,...,...,...,...,...
438,2024-02-15,7035163,Email Concern,Forgot password,Open,BDD,Medium,NaT,NaN
439,2024-02-15,7037058,Software/App Installation,Excel Problem - file can't be saved,Resolved,LEGAL,Low,2024-02-15,File save in autorecovery
440,2024-02-16,7039343,DPC Module Remote Desktop,DPC Remote Desktop - Bacolod 02/16/2024,Resolved,FINANCE,Low,2024-02-16,coordinate to HQ group to restart the module
441,2024-02-16,7039580,DPC Module Remote Desktop,Unable to access remote module IP 192.168.20.1...,Resolved,FINANCE,Low,2024-02-16,coordinate to HQ group to restart the module


In [ ]:
engine = connect_to_postgres()

In [ ]:
df.to_sql(name="tickets_temp", con=engine, if_exists="replace", index=False)

443

In [ ]:
print(pd.io.sql.get_schema(df, name="tickets_temp"))

CREATE TABLE "tickets_temp" (
"created_date" TIMESTAMP,
  "ticket_id" INTEGER,
  "category_name" TEXT,
  "Subject" TEXT,
  "Status" TEXT,
  "department_name" TEXT,
  "Urgency" TEXT,
  "resolved_date" TIMESTAMP,
  "remarks" TEXT
)
